Вариант 3. Построение модели и оптимизация гиперпараметров.
Данный вариант предполагает фокусировку на процессе улучшения эффективности модели обучения с учителем. Студенту следует подготовить датасет к обучению, обучить одну из моделей с учителем со значениями гиперпараметров по умолчанию, получить значение эффективности. После этого вручную или автоматически подобрать значения гиперпараметров таким образом, чтобы получить максимальный прирост эффективности.

### Загрузка данных

In [1]:
import pandas as pd
import warnings
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score, f1_score, precision_score, recall_score
#если классификатор:
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
#если регрессия:
#from sklearn.linear_model import LinearRegression
#from sklearn.neighbors import KNeighborsRegressor
#from sklearn.tree import DecisionTreeRegressor
#from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
#from sklearn.metrics import make_scorer, r2_score, mean_absolute_error, mean_absolute_percentage_error, mean_squared_error

In [2]:
warnings.filterwarnings("ignore")

In [3]:
df = pd.read_csv('train_loan.csv')[:10000]
df

,Loan_ID,Loan_Amount_Requested,Length_Employed,Home_Owner,Annual_Income,Income_Verified,Purpose_Of_Loan,Debt_To_Income,Inquiries_Last_6Mo,Months_Since_Deliquency,Number_Open_Accounts,Total_Accounts,Gender,Interest_Rate
0,10000001,"7,000",< 1 year,Rent,68000.0,not verified,car,18.37,0,NaN,9,14,Female,1
1,10000002,"30,000",4 years,Mortgage,NaN,VERIFIED - income,debt_consolidation,14.93,0,17.0,12,24,Female,3
2,10000003,"24,725",7 years,Mortgage,75566.4,VERIFIED - income source,debt_consolidation,15.88,0,NaN,12,16,Male,3
3,10000004,"16,000",< 1 year,NaN,56160.0,VERIFIED - income source,debt_consolidation,14.34,3,NaN,16,22,Male,3
4,10000005,"17,000",8 years,Own,96000.0,VERIFIED - income source,debt_consolidation,22.17,1,NaN,19,30,Female,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,10009996,"15,000",10+ years,NaN,55500.0,VERIFIED - income source,credit_card,27.18,0,NaN,10,20,Male,1
9996,10009997,"21,000",10+ years,Mortgage,58000.0,VERIFIED - income source,credit_card,11.38,0,NaN,5,15,Female,1
9997,10009998,"5,000",NaN,Rent,24000.0,VERIFIED - income,debt_consolidation,29.30,0,61.0,17,29,Female,2
9998,10009999,"21,000",10+ years,Mortgage,NaN,VERIFIED - income,debt_consolidation,23.63,3,NaN,22,40,Male,2


In [4]:
#сократили размер датасета, тк он был огромный, это мешало выполнению работы, тк занимало очень много времени

### Описание датасета

В данном датасете представлены данные о заемщиках, на основании которых кредиторы определяют процентную ставку по кредиту. Решаемая задача небинарной классификации. Цель построения модели - предсказать для заемщика одну из процентных ставок: 1%, 2%, 3%.

### Анализ и обработка данных

In [5]:
df.shape

(10000, 14)

В датасете присутствует 164309 точек данных (объектов, заемщиков), 14 переменных. 

In [6]:
df.isna().sum() / df.shape[0] * 100

Loan_ID                     0.00
Loan_Amount_Requested       0.00
Length_Employed             4.61
Home_Owner                 15.77
Annual_Income              15.27
Income_Verified             0.00
Purpose_Of_Loan             0.00
Debt_To_Income              0.00
Inquiries_Last_6Mo          0.00
Months_Since_Deliquency    53.35
Number_Open_Accounts        0.00
Total_Accounts              0.00
Gender                      0.00
Interest_Rate               0.00
dtype: float64

Ячейкой выше представлено процентное соотношение пропущенных значений. В переменных Length_Employed, Home_Owner, Annual_Income, Months_Since_Deliquency пропущены значения. 

Определим типы переменных:
* Length_Employed - категориальная 
* Home_Owner - категориальная
* Annual_Income - числовая
* Months_Since_Deliquency - числовая

Пропущенные значения в переменных категориального типа можно заменить модой (самое популярное значение), либо удалить, либо заменить на "значение неизвестно" ("unknown"), либо как-то иначе. Пропущенные значения в переменных числового типа можно заменить медианой, средним значением, либо как-то иначе. 

In [7]:
# Пропущенные значения переменной Length_Employed (категориальная) удалим
# тк они встречается в 4% от всех значений этой переменной

df = df.dropna(subset=['Length_Employed'])
df

,Loan_ID,Loan_Amount_Requested,Length_Employed,Home_Owner,Annual_Income,Income_Verified,Purpose_Of_Loan,Debt_To_Income,Inquiries_Last_6Mo,Months_Since_Deliquency,Number_Open_Accounts,Total_Accounts,Gender,Interest_Rate
0,10000001,"7,000",< 1 year,Rent,68000.0,not verified,car,18.37,0,NaN,9,14,Female,1
1,10000002,"30,000",4 years,Mortgage,NaN,VERIFIED - income,debt_consolidation,14.93,0,17.0,12,24,Female,3
2,10000003,"24,725",7 years,Mortgage,75566.4,VERIFIED - income source,debt_consolidation,15.88,0,NaN,12,16,Male,3
3,10000004,"16,000",< 1 year,NaN,56160.0,VERIFIED - income source,debt_consolidation,14.34,3,NaN,16,22,Male,3
4,10000005,"17,000",8 years,Own,96000.0,VERIFIED - income source,debt_consolidation,22.17,1,NaN,19,30,Female,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,10009995,"8,000",10+ years,NaN,78000.0,VERIFIED - income source,debt_consolidation,2.80,2,71.0,15,35,Male,2
9995,10009996,"15,000",10+ years,NaN,55500.0,VERIFIED - income source,credit_card,27.18,0,NaN,10,20,Male,1
9996,10009997,"21,000",10+ years,Mortgage,58000.0,VERIFIED - income source,credit_card,11.38,0,NaN,5,15,Female,1
9998,10009999,"21,000",10+ years,Mortgage,NaN,VERIFIED - income,debt_consolidation,23.63,3,NaN,22,40,Male,2


In [8]:
# Неизвестные значения переменной Home_Owner заменяем на "None"

df = df.fillna({'Home_Owner': 'None'})

In [9]:
# Неизвестные значения переменной Annual_Income заменяем средним

df['Annual_Income'] = df['Annual_Income'].fillna(df['Annual_Income'].mean())

In [10]:
# Неизвестные значения переменной Months_Since_Deliquency заменяем средним

df['Months_Since_Deliquency'] = df['Months_Since_Deliquency'].fillna(df['Months_Since_Deliquency'].mean())

In [11]:
#проверка
df.isna().sum() 

Loan_ID                    0
Loan_Amount_Requested      0
Length_Employed            0
Home_Owner                 0
Annual_Income              0
Income_Verified            0
Purpose_Of_Loan            0
Debt_To_Income             0
Inquiries_Last_6Mo         0
Months_Since_Deliquency    0
Number_Open_Accounts       0
Total_Accounts             0
Gender                     0
Interest_Rate              0
dtype: int64

In [12]:
#Удаляем переменные, которые не нужны для построения моделей

df = df.drop(columns=['Loan_ID'])
df

,Loan_Amount_Requested,Length_Employed,Home_Owner,Annual_Income,Income_Verified,Purpose_Of_Loan,Debt_To_Income,Inquiries_Last_6Mo,Months_Since_Deliquency,Number_Open_Accounts,Total_Accounts,Gender,Interest_Rate
0,"7,000",< 1 year,Rent,68000.000000,not verified,car,18.37,0,34.66278,9,14,Female,1
1,"30,000",4 years,Mortgage,74744.389281,VERIFIED - income,debt_consolidation,14.93,0,17.00000,12,24,Female,3
2,"24,725",7 years,Mortgage,75566.400000,VERIFIED - income source,debt_consolidation,15.88,0,34.66278,12,16,Male,3
3,"16,000",< 1 year,None,56160.000000,VERIFIED - income source,debt_consolidation,14.34,3,34.66278,16,22,Male,3
4,"17,000",8 years,Own,96000.000000,VERIFIED - income source,debt_consolidation,22.17,1,34.66278,19,30,Female,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,"8,000",10+ years,None,78000.000000,VERIFIED - income source,debt_consolidation,2.80,2,71.00000,15,35,Male,2
9995,"15,000",10+ years,None,55500.000000,VERIFIED - income source,credit_card,27.18,0,34.66278,10,20,Male,1
9996,"21,000",10+ years,Mortgage,58000.000000,VERIFIED - income source,credit_card,11.38,0,34.66278,5,15,Female,1
9998,"21,000",10+ years,Mortgage,74744.389281,VERIFIED - income,debt_consolidation,23.63,3,34.66278,22,40,Male,2


In [13]:
# Заменили тип данных у переменной Loan_Amount_Requested
df['Loan_Amount_Requested'] = df['Loan_Amount_Requested'].astype(str).str.replace(',', '').astype(int)
df

,Loan_Amount_Requested,Length_Employed,Home_Owner,Annual_Income,Income_Verified,Purpose_Of_Loan,Debt_To_Income,Inquiries_Last_6Mo,Months_Since_Deliquency,Number_Open_Accounts,Total_Accounts,Gender,Interest_Rate
0,7000,< 1 year,Rent,68000.000000,not verified,car,18.37,0,34.66278,9,14,Female,1
1,30000,4 years,Mortgage,74744.389281,VERIFIED - income,debt_consolidation,14.93,0,17.00000,12,24,Female,3
2,24725,7 years,Mortgage,75566.400000,VERIFIED - income source,debt_consolidation,15.88,0,34.66278,12,16,Male,3
3,16000,< 1 year,None,56160.000000,VERIFIED - income source,debt_consolidation,14.34,3,34.66278,16,22,Male,3
4,17000,8 years,Own,96000.000000,VERIFIED - income source,debt_consolidation,22.17,1,34.66278,19,30,Female,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,8000,10+ years,None,78000.000000,VERIFIED - income source,debt_consolidation,2.80,2,71.00000,15,35,Male,2
9995,15000,10+ years,None,55500.000000,VERIFIED - income source,credit_card,27.18,0,34.66278,10,20,Male,1
9996,21000,10+ years,Mortgage,58000.000000,VERIFIED - income source,credit_card,11.38,0,34.66278,5,15,Female,1
9998,21000,10+ years,Mortgage,74744.389281,VERIFIED - income,debt_consolidation,23.63,3,34.66278,22,40,Male,2


### Обучение моделей и сравнение эффективности

Кодирование категориальных признаков с помощью приведения их к бинарной шкале

In [14]:
df = pd.get_dummies(df)
df

,Loan_Amount_Requested,Annual_Income,Debt_To_Income,Inquiries_Last_6Mo,Months_Since_Deliquency,Number_Open_Accounts,Total_Accounts,Interest_Rate,Length_Employed_1 year,Length_Employed_10+ years,...,Purpose_Of_Loan_major_purchase,Purpose_Of_Loan_medical,Purpose_Of_Loan_moving,Purpose_Of_Loan_other,Purpose_Of_Loan_renewable_energy,Purpose_Of_Loan_small_business,Purpose_Of_Loan_vacation,Purpose_Of_Loan_wedding,Gender_Female,Gender_Male
0,7000,68000.000000,18.37,0,34.66278,9,14,1,0,0,...,0,0,0,0,0,0,0,0,1,0
1,30000,74744.389281,14.93,0,17.00000,12,24,3,0,0,...,0,0,0,0,0,0,0,0,1,0
2,24725,75566.400000,15.88,0,34.66278,12,16,3,0,0,...,0,0,0,0,0,0,0,0,0,1
3,16000,56160.000000,14.34,3,34.66278,16,22,3,0,0,...,0,0,0,0,0,0,0,0,0,1
4,17000,96000.000000,22.17,1,34.66278,19,30,1,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,8000,78000.000000,2.80,2,71.00000,15,35,2,0,1,...,0,0,0,0,0,0,0,0,0,1
9995,15000,55500.000000,27.18,0,34.66278,10,20,1,0,1,...,0,0,0,0,0,0,0,0,0,1
9996,21000,58000.000000,11.38,0,34.66278,5,15,1,0,1,...,0,0,0,0,0,0,0,0,1,0
9998,21000,74744.389281,23.63,3,34.66278,22,40,2,0,1,...,0,0,0,0,0,0,0,0,0,1


In [15]:
#Выделим целевую переменную (y) и признаки (X)
X = df.drop(columns=['Interest_Rate'])
y = df['Interest_Rate']

#с помощью библиотеки sklearn разделили исходную выборку на тренировчную(80%) и тестовую(20%):
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

Будем использовать следующий классификатор: метод k ближайших соседей (KNeighborsClassifier). 

In [16]:
#если регрессия, то модель регресии

In [17]:
cls_knn = KNeighborsClassifier()
#если регрессия:
#reg_knn = KNeighborsRegressor()

In [18]:
# Для оценки эффективности классификации будем использовать стандартные метрики
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score, average='macro'), # тк больше 2 классов, то average='macro'
    'recall': make_scorer(recall_score, average='macro'),
    'f1': make_scorer(f1_score, average='macro'),
}
#если регрессия:
#scoring = {
#    'r2': make_scorer(r2_score),
#    'rmse': make_scorer(mean_squared_error, squared=False),
#    'mae': make_scorer(mean_absolute_error),
#    'mape': make_scorer(mean_absolute_percentage_error)
#}

Выведем метрики качества для всех моделей. Метрики качества получены путем проведения кросс валидации.
Мы используем кросс валидацию, чтобы оценить качество работы модели и сравнить модели между собой.

Кросс-валидация — это методика обучения и оценки модели, которая разбивает данные на несколько секций и обучает модель на этих секциях.

In [19]:
scores = {}
#если регрессия, то:
# for score_name, values in cross_validate(reg_knn, X_train, y_train, scoring=scoring).items():
for score_name, values in cross_validate(cls_knn, X_train, y_train, scoring=scoring).items():
    scores[score_name] = values.mean()
pd.Series(scores)

fit_time          0.007539
score_time        0.091742
test_accuracy     0.394574
test_precision    0.378128
test_recall       0.372163
test_f1           0.369043
dtype: float64

Зафиксиурем метрики для стандартных гиперпараметров.

In [20]:
#если регрессия, то смотрим на r^2 и mape (ошибка в процентах). 
#если классификатор, то f1

### Подбор гиперпараметров

Будем перебирать гиперпараметры: n_neighbors, weights и p у классификатора KNeighborsClassifier. Для этого используем GridSearchCV. Он позволяет путем перебора найти такие гиперпараметры, при которых модель показывает наилучшую эффективность. При этом рассчет показателей качества модели производится также с помощью кросс-валидации.

In [21]:
parameters = {
    'n_neighbors': [3, 5, 10, 50], 
    'weights': ['uniform', 'distance'],
    'p': [10e-5, 1, 10]
            }

In [22]:
#если регрессия, то:
#best_params = GridSearchCV(reg_knn, parameters).fit(X_train, y_train).best_params_
best_params = GridSearchCV(cls_knn, parameters).fit(X_train, y_train).best_params_
best_params

{'n_neighbors': 50, 'p': 1, 'weights': 'uniform'}

In [23]:
#если регрессия
#reg_knn = KNeighborsRegressor(**best_params)
cls_knn = KNeighborsClassifier(**best_params)
scores = {}
#если регрессия, то:
# for score_name, values in cross_validate(reg_knn, X_train, y_train, scoring=scoring).items():
for score_name, values in cross_validate(cls_knn, X_train, y_train, scoring=scoring).items():
    scores[score_name] = values.mean()
pd.Series(scores)

fit_time          0.007230
score_time        0.134876
test_accuracy     0.456951
test_precision    0.402599
test_recall       0.380000
test_f1           0.349684
dtype: float64

Стандартные параметры по умолчанию оказались лучше, тк метрики качества модели стали хуже

In [31]:
#смотреть на f1 или r^2
#Подобранные параметры оказались лучше, тк метрики качества модели стали лучше

### Построение итоговой модели

In [ ]:
#если метрики качества модели стали лучше, то
# cls_knn = KNeighborsClassifier(**best_params).fit(X_train, y_train)

#если регрессия
#reg_knn = KNeighborsRegressor(**best_params).fit(X_train, y_train)

In [42]:
#создадим классификатор с подобранными параметрами и обучим его на обучающей выборке
cls_knn = KNeighborsClassifier().fit(X_train, y_train)

In [43]:
#делаем предсказания по нашему классификатору на тестовой выборке
y_pred = cls_knn.predict(X_test)

Выведем метрики эффективности и сделаем вывод о применимости модели

In [44]:
accuracy_score(y_test, y_pred)

0.3967505241090147

In [45]:
precision_score(y_test, y_pred, average='macro')

0.38297048966144054

In [46]:
recall_score(y_test, y_pred, average='macro')

0.37779936929429586

In [ ]:
f1_score(y_test, y_pred, average='macro')

In [29]:
#если регрессия
#r2_score(y_test, y_pred)
#mean_squared_error(y_test, y_pred, squared=False)
#mean_absolute_error(y_test, y_pred)
#mean_absolute_percentage_error(y_test, y_pred)

### Показатели метрик плохие. Модель не применима 